In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Extracting ex-post Index rebalancing results

In order to determine how effective our trading strategy is, we shall compare our results with the ex-post.<br>

Here, we compare the index before the rank date and after the effective date. We shall identify which stock is included or excluded from the FTSE100, and if it was excluded from the FTSE100 did it get included into the FTSE250. Stocks that were excluded from either of the index but did not get included into the other may mean that:
- the market cap of the stock has gotten lower than the requirement to be qualified into FTSE250, or
- the stock has been delisted

In [2]:
# date pairs
date_pairs =\
[
    ('20220524', '20220620'),
    ('20220823', '20220920'),
    ('20221122', '20221219'),
    ('20230221', '20230320'),
    ('20230523', '20230619')
]


In [3]:
def get_constituent_changes(index, date_pair):
    pre_review, post_review = date_pair
    
    # Extract dataframe for pre-review data
    df_pre_review =\
    (
        pd
        .read_excel(
            f"../FTSE/{index}_pre-review.xlsx", 
            sheet_name = pre_review
        )
        .set_index(['Ticker', 'Name','SEDOL\n', 'ISIN\n'])
        .drop(columns = ['Weight', 'Shares', 'Price'])
    )
    
    df_pre_review.columns = ['Market Cap (Pre)']
    
    # Extract dataframe for post-review data
    
    df_post_review =\
    (
        pd
        .read_excel(
            f"../FTSE/{index}_post-review.xlsx", 
            sheet_name = post_review
        )
        .set_index(['Ticker', 'Name','SEDOL\n', 'ISIN\n'])
        .drop(columns = ['Weight', 'Shares', 'Price'])
    )
    
    df_post_review.columns = ['Market Cap (Post)']
    
    # Derive constituent changes
    df_comb =\
    (
        pd
        .merge(
            df_pre_review,
            df_post_review,
            how = 'outer',
            left_index = True,
            right_index = True,
        )
    )
    
    df_comb['Post Date'] =\
    (
        datetime
        .strptime(
            post_review, 
            "%Y%m%d"
        )
    )
    
    df_const_chng  =\
    (
        df_comb[df_comb
                .isna()
                .any(axis=1)]
    )
    
    
    if not df_const_chng.empty:
    
        df_const_chng[index] =\
        (
            df_const_chng
            .apply(
                lambda x: 
                -1 if pd.isnull(x['Market Cap (Post)']) else 1,
                axis = 1
            )
        )
    
    return df_const_chng 

In [4]:
def get_FTSE100_changes(date_pair):
    indices =\
    {
        "UKX" : "FTSE100",
        "MCX" : "FTSE250"
    }
    
    ftse100 = get_constituent_changes('UKX', date_pair)
    ftse250 = get_constituent_changes('MCX', date_pair)
    
    df =\
    (
        pd
        .merge(
            ftse100,
            ftse250,
            how = 'outer',
            left_index = True,
            right_index = True
        )
    )
    
    df =\
    (
        df
        .rename(columns=indices)
    )
    
    df.columns =\
    [
        col.replace('\n','') for col in df.columns 
    ]
    
    for col in ['Market Cap (Pre)', 'Market Cap (Post)', 'Post Date']:
        df[f'{col}_x'] = df[f'{col}_x'].fillna(0)
        df[col] = df.apply(lambda x: x[f'{col}_x'] if x[f'{col}_x'] else x[f'{col}_y'], axis = 1)
    
    df =\
    (
        df
        .drop([
            'Market Cap (Pre)_x', 
            'Market Cap (Post)_x', 
            'Post Date_x', 
            'Market Cap (Pre)_y', 
            'Market Cap (Post)_y', 
            'Post Date_y'],
        axis = 1)
    )
        
    return df
    

In [5]:
ftseAll_changes = pd.concat(map(get_FTSE100_changes, date_pairs))

ftseAll_changes\
.to_csv(
    "../output/ftse_largemid_rebalancing_summary.csv"
)

In [6]:
ftseAll_changes

,,,,FTSE100,FTSE250,Market Cap (Pre),Market Cap (Post),Post Date
Ticker,Name,SEDOL,ISIN,,,,,
ASC LN Equity,ASOS PLC,3092725,GB0030927254,NaN,1.0,NaN,870979136,2022-06-20
BGEO LN Equity,Bank of Georgia Group PLC,BF4HYT8,GB00BF4HYT85,NaN,1.0,NaN,738524800,2022-06-20
CLG LN Equity,GXO Logistics UK II Ltd,BMMV6B7,GB00BMMV6B79,NaN,-1.0,874229248,NaN,2022-06-20
CNA LN Equity,Centrica PLC,B033F22,GB00B033F229,1.0,-1.0,4912636928,4763841536,2022-06-20
IDS LN Equity,International Distributions Services PLC,BDVZYZ7,GB00BDVZYZ77,-1.0,1.0,2999578880,2687859968,2022-06-20
...,...,...,...,...,...,...,...,...
MEGP LN Equity,ME GROUP INTERNATIONAL PLC,0848125,GB0008481250,NaN,1.0,NaN,608598720,2023-06-19
NAS LN Equity,North Atlantic Smaller Cos Investment Trust PLC,0643900,GB0006439003,NaN,1.0,NaN,--,2023-06-19
TLW LN Equity,Tullow Oil PLC,0150080,GB0001500809,NaN,-1.0,361791968.0,NaN,2023-06-19
